<a href="https://colab.research.google.com/github/karsarobert/ChatGPT2025/blob/main/09/ChatGPT2025_09_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Chat GPT és más nagy nyelvi modellek alkalmazása
##PTE Gépi tanulás
###9. gyakorlat: Langchain 1. rész
2025. április 7.

Bevezetés és Célkitűzés

Mi a LangChain?
A LangChain egy keretrendszer, amelyet arra terveztek, hogy megkönnyítse a nagy nyelvi modelleket (LLM-eket) használó alkalmazások fejlesztését. Lehetővé teszi komponensek (pl. modellek, promptok, adatforrások, memóriakezelés) összekapcsolását és "láncokká" (chains) fűzését komplexebb munkafolyamatok létrehozása érdekében.
A gyakorlat célja:
Ez a gyakorlat bemutatja a LangChain alapvető építőköveit: hogyan kommunikáljunk LLM-ekkel, hogyan kezeljük a promptokat, hogyan töltsünk be és dolgozzunk fel külső dokumentumokat, hogyan használjunk vektortárolókat a releváns információk keresésére, és hogyan építsünk egyszerű láncokat ezekből a komponensekből.
Előfeltételek:
Alapvető Python programozási ismeretek.
Hozzáférési kulcsok (API Keys) az OpenAI és/vagy a Hugging Face szolgáltatásaihoz. Ezeket a megfelelő platformokon lehet generálni.
2. Környezet Előkészítése (Telepítés és API Kulcsok)

Szükséges Csomagok Telepítése:
A LangChain és a hozzá kapcsolódó könyvtárak telepítéséhez futtassuk a következő parancsot a Python környezetünkben (pl. egy Colab notebook cellájában):

In [36]:
# source: 2
!pip install langchain openai tiktoken faiss-cpu sentence-transformers langchain_community langchain_openai
# source: 4

Magyarázat:
langchain: A fő LangChain keretrendszer.
openai: Az OpenAI API-jának használatához szükséges könyvtár (GPT modellekhez).
tiktoken: Az OpenAI által használt tokenizáló könyvtár, segít a szövegek tokenekre bontásában és a hosszuk mérésében.
faiss-cpu: Egy könyvtár a hatékony hasonlósági kereséshez és sűrű vektorok klaszterezéséhez (CPU-verzió). Ezt használjuk a vektortároló implementálásához.
sentence-transformers: Népszerű könyvtár state-of-the-art mondat-, szöveg- és képbeágyazások készítéséhez (Hugging Face modellekhez).
langchain_community: A LangChain közösségi integrációit és komponenseit tartalmazó csomag.

In [37]:
# source: 9
from google.colab import userdata

openai_api_key = userdata.get('openai_api')
huggingface_api_token = userdata.get('HF_TOKEN')

# Használat később:
# llm = OpenAI(api_key=openai_api_key)
# llm_hf = HuggingFaceHub(huggingfacehub_api_token=huggingface_api_token, ...)

Helyi fejlesztés esetén környezeti változók vagy .env fájlok használata ajánlott (pl. a python-dotenv csomaggal).
3. LangChain Alapkoncepciók és Komponensek

Modellek (LLMs)

Leírás: Ezek a nagy nyelvi modellek, amelyekkel interakcióba lépünk. A LangChain egységes interfészt biztosít különböző modellekhez (OpenAI, Hugging Face Hub, stb.).
Példa (OpenAI):

In [38]:
# source: 9
from google.colab import userdata
from langchain_openai import OpenAI

llm = OpenAI(api_key=userdata.get('openai_api'))

response = llm.invoke('tell me a joke')
print(response)




Why was the math book sad?

Because it had too many problems!


Magyarázat: Létrehozunk egy OpenAI objektumot az API kulcsunkkal. Ezt az objektumot használhatjuk szöveges promptok küldésére a modellnek és válaszok fogadására. Fontos figyelni az elavulásra vonatkozó figyelmeztetésekre (LangChainDeprecationWarning) és lehetőség szerint a frissebb API-kat használni (langchain_openai csomag, invoke metódus).
Prompt Sablonok (Prompt Templates)

Leírás: Segítenek dinamikusan és strukturáltan létrehozni a nyelvi modelleknek szánt utasításokat (promptokat). Lehetővé teszik változók beillesztését előre definiált szövegsablonokba.  Függetlenek a konkrét LLM-től.
Példa:

In [39]:
# source: 13
from langchain import PromptTemplate

template = """Sentence: {sentence}
Translation in {language}:"""

prompt = PromptTemplate(template=template, input_variables=["sentence", "language"])

formatted_prompt = prompt.format(sentence="the cat is on the table", language="hungarian")
print(formatted_prompt)
# Kimenet:
# Sentence: the cat is on the table
# Translation in hungarian:

Sentence: the cat is on the table
Translation in hungarian:


Magyarázat: Definiálunk egy template sztringet {} jelekkel a változók helyén. A PromptTemplate objektum megmondja, mely változók (input_variables) várhatók. A .format() metódussal helyettesítjük be az aktuális értékeket.

Befejezési vs. Chat Modellek:
Befejezési (Completion) modellek: Egyetlen szöveges bemenetet (prompt) kapnak, és megpróbálják azt folytatni vagy arra válaszolni egyetlen szöveges kimenettel.

Chat modellek: Üzenetek listáját kapják bemenetként, ahol minden üzenetnek van egy szerepe (pl. system, human, ai) és tartalma. A válaszuk általában egy új 'ai' szerepű üzenet.  A prompt sablonok mindkettőhöz használhatók, de a chat modellekhez speciális chat-prompt sablonok is léteznek.
Példaválasztó (Example Selector)

Leírás: Olyan komponens, amely lehetővé teszi, hogy a promptba dinamikusan illesszünk be releváns példákat (input/output párokat) a feladat jobb szemléltetésére vagy a kimeneti formátum meghatározására ("few-shot prompting").
 (Megjegyzés: A jegyzetfüzet említi, de nem tartalmaz konkrét példát a használatára.)
Dokumentum Betöltők (Document Loaders)

Leírás: Adatok betöltésére szolgálnak különböző forrásokból (CSV, PDF, HTML, szövegfájlok stb.) és Document objektumokká alakítására.  Egy Document objektum tartalmazza a szövegrészletet (page_content) és a hozzá tartozó metaadatokat (metadata).
Példa (CSV Loader):
Előkészítés (Adatgenerálás yfinance-szel):

In [40]:
# source: 22
import yfinance as yf
import pandas as pd

tickers = ['AAPL']
start_date = '2020-01-01'
end_date = '2022-12-31'
data = pd.DataFrame()

for ticker in tickers:
    df = yf.download(ticker, start=start_date, end=end_date, interval='1mo')
    df['Ticker'] = ticker
    data = pd.concat([data, df])

# source: 23
data.reset_index(inplace=True)
data.to_csv('stock_data.csv', index=False)
print("stock_data.csv létrehozva.")

[*********************100%***********************]  1 of 1 completed

stock_data.csv létrehozva.


In [41]:
# source: 27
from langchain_community.document_loaders.csv_loader import CSVLoader # langchain_community használata javasolt

loader = CSVLoader(file_path='stock_data.csv')
data_docs = loader.load()

# Az első betöltött dokumentum tartalmának és metaadatainak megtekintése
print(data_docs[0])
# Várható kimenet (részlet):
# Document(page_content='Date: 2020-01-01 00:00:00\nClose: 77.16500091552734\n...Ticker: AAPL', metadata={'source': 'stock_data.csv', 'row': 0})

page_content='Date: 
Close: AAPL
High: AAPL
Low: AAPL
Open: AAPL
Volume: AAPL
Ticker: ' metadata={'source': 'stock_data.csv', 'row': 0}


Magyarázat: A CSVLoader betölti a megadott CSV fájlt. Alapértelmezetten minden sor egy külön Document objektum lesz, ahol a page_content a sor tartalmát tartalmazza "kulcs: érték" formában, a metadata pedig a forrást és a sor számát.
Dokumentum Átalakítók (Document Transformers - Text Splitters)

Leírás: A betöltött dokumentumok módosítására szolgálnak, leggyakrabban hosszú szövegek kisebb, kezelhető darabokra ("chunks") vágására, hogy azok beleférjenek a modell kontextusablakába anélkül, hogy fontos információ veszne el.
Példa (RecursiveCharacterTextSplitter):
Előkészítés (Szövegfájl letöltése és betöltése):

In [42]:
!gdown 183bdW5i6KCv5fq_nZGx3nOIi6N70gZ0x

Downloading...
From: https://drive.google.com/uc?id=183bdW5i6KCv5fq_nZGx3nOIi6N70gZ0x
To: /content/piszkosfred_reszlet.txt
100% 5.22k/5.22k [00:00<00:00, 7.96MB/s]


In [43]:
# source: 33
# !gdown 183bdW5i6KCv5fq_nZGx3n0Ii6N70gZ0x # Fájl letöltése Google Drive-ról (ha szükséges)

# source: 34
from langchain_community.document_loaders import TextLoader # langchain_community használata javasolt

loader = TextLoader("piszkosfred_reszlet.txt") # Feltételezi, hogy a fájl létezik
text_document = loader.load()

# print(text_document) # A teljes betöltött dokumentum (egy elemű lista)

In [44]:
# source: 34
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,      # A darabok maximális mérete (karakterben)
    chunk_overlap=50,     # Átfedés a darabok között (kontextus megőrzése)
    length_function=len,  # Hogyan mérjük a darab hosszát (itt karakterek száma)
)

texts = text_splitter.split_documents(text_document)

# Az első néhány darab megtekintése
print("Első darab:")
print(texts[0])
# source: 35
print("\nMásodik darab:")
print(texts[1])
# source: 36
print("\nHarmadik darab:")
print(texts[2])
# source: 37

# Várható kimenet (példák):
# Első darab:
# page_content='Rejtő Jenő\n\nPiszkos Fred, a kapitány' metadata={'source': 'piszkosfred_reszlet.txt'}
#
# Második darab:
# page_content='ELSŐ FEJEZET\n\n1\n\nUram! A késemért jöttem!\n\nHol hagyta?\n\nValami matrózban.\n\nMilyen kés volt?\n\nAcél. Keskeny penge, kissé hajlott. Nem látta?' metadata={'source': 'piszkosfred_reszlet.txt'}
#
# Harmadik darab:
# page_content='Acél. Keskeny penge, kissé hajlott. Nem látta?\n\nVárjunk... Csak lassan, kérem... Milyen volt a nyele?\n\nKagyló.\n\nHány részből?\n\nEgy darabból készült.\n\nAkkor nincs baj. Megvan a kés!\n\nHol?' metadata={'source': 'piszkosfred_reszlet.txt'}

Első darab:
page_content='﻿
Rejtő Jenő

Piszkos Fred, a kapitány' metadata={'source': 'piszkosfred_reszlet.txt'}

Második darab:
page_content='ELSŐ FEJEZET
1
- Uram! A késemért jöttem!
- Hol hagyta?
- Valami matrózban.
- Milyen kés volt?
- Acél. Keskeny penge, kissé hajlott. Nem látta?' metadata={'source': 'piszkosfred_reszlet.txt'}

Harmadik darab:
page_content='- Acél. Keskeny penge, kissé hajlott. Nem látta?
- Várjunk... Csak lassan, kérem... Milyen volt a nyele?
- Kagyló.
- Hány részből?
- Egy darabból készült.
- Akkor nincs baj. Megvan a kés!
- Hol?' metadata={'source': 'piszkosfred_reszlet.txt'}


Magyarázat: A RecursiveCharacterTextSplitter megpróbálja a szöveget logikai egységek mentén (pl. bekezdések, sorok, mondatok) darabolni, amíg a chunk_size alá nem kerül a méret. A chunk_overlap segít abban, hogy a darabok közötti vágásnál ne vesszen el kontextus.
Szövegbeágyazó Modellek (Text Embedding Models)

Leírás: Ezek a modellek a szöveget (szavakat, mondatokat, dokumentumrészleteket) sűrű numerikus vektorokká alakítják (ún. "embedding" vagy beágyazás). A hasonló jelentésű szövegek vektorai közel lesznek egymáshoz a vektortérben. Ez kulcsfontosságú a szemantikus kereséshez és az LLM-ek "külső tudással" való kiegészítéséhez.
Példa (OpenAI):

In [45]:
# source: 41
from google.colab import userdata
from langchain_openai import OpenAIEmbeddings # Javasolt import

embeddings_model = OpenAIEmbeddings(
    model='text-embedding-ada-002',
    api_key=userdata.get('openai_api')
)

# Több dokumentum beágyazása
documents_to_embed = [
    "Good morning!",
    "Oh, hello!",
    "I want to report an accident",
    "Sorry to hear that. May I ask your name?",
    "Sure, Mario Rossi."
]
embeddings = embeddings_model.embed_documents(documents_to_embed)

print("Dokumentum beágyazások:")
# source: 42
print(f"Vektorok száma: {len(embeddings)}; Egy vektor dimenziója: {len(embeddings[0])}")

# Egy lekérdezés beágyazása
query = "What was the name mentioned in the conversation?"
embedded_query = embeddings_model.embed_query(query)

print("\nLekérdezés beágyazása:")
print(f"Vektor dimenziója: {len(embedded_query)}")
print(f"Minta a vektor első 5 eleméből: {embedded_query[:5]}")

Dokumentum beágyazások:
Vektorok száma: 5; Egy vektor dimenziója: 1536

Lekérdezés beágyazása:
Vektor dimenziója: 1536
Minta a vektor első 5 eleméből: [0.005329647101461887, -0.0006122003542259336, 0.0389961302280426, -0.002898985054343939, -0.008904732763767242]


In [58]:
# source: 43
from langchain_community.embeddings import HuggingFaceEmbeddings # Javasolt import

# Explicit modellnév megadása javasolt lehet: HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')

embeddings_modelHF = HuggingFaceEmbeddings(model_name='karsar/paraphrase-multilingual-MiniLM-L12-hu-v3') # Letölti az alapértelmezett modellt

embeddingsHF = embeddings_modelHF.embed_documents(documents_to_embed)

print("Dokumentum beágyazások (Hugging Face):")
print(f"Vektorok száma: {len(embeddingsHF)}; Egy vektor dimenziója: {len(embeddingsHF[0])}")

embedded_queryHF = embeddings_modelHF.embed_query(query)

print("\nLekérdezés beágyazása (Hugging Face):")
print(f"Vektor dimenziója: {len(embedded_queryHF)}")
print(f"Minta a vektor első 5 eleméből: {embedded_queryHF[:5]}")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/573k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Dokumentum beágyazások (Hugging Face):
Vektorok száma: 5; Egy vektor dimenziója: 384

Lekérdezés beágyazása (Hugging Face):
Vektor dimenziója: 384
Minta a vektor első 5 eleméből: [-0.36755359172821045, 0.798269510269165, 0.09128328412771225, -0.22096721827983856, -0.16855275630950928]


Magyarázat: Az embed_documents metódus listányi szöveget, míg az embed_query egyetlen szöveget alakít vektorrá. A különböző beágyazó modellek eltérő dimenziójú vektorokat eredményezhetnek (pl. az ada-002 1536 dimenziósat, a HuggingFace alapmodell ettől eltérhet).
Vektortárolók (Vector Stores)

Leírás: Olyan adatbázisok, amelyek kifejezetten a szövegbeágyazások (vektorok) tárolására és a hozzájuk legközelebb álló vektorok (és így a legrelevánsabb szövegek) gyors keresésére vannak optimalizálva.  Ez teszi lehetővé a szemantikus keresést nagy mennyiségű szöveges adatban.
Példa (FAISS):
Előkészítés (Szövegfájl letöltése, ha szükséges):

In [47]:
# source: 47
!gdown 1tICd5l-gv1PhZgvCkLD4ydqq-jIaHUv-

Downloading...
From: https://drive.google.com/uc?id=1tICd5l-gv1PhZgvCkLD4ydqq-jIaHUv-
To: /content/dialog.txt
100% 114/114 [00:00<00:00, 364kB/s]


In [59]:
# source: 48
from google.colab import userdata
from langchain_community.document_loaders import TextLoader # Javasolt import
from langchain_openai import OpenAIEmbeddings             # Javasolt import
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS        # Javasolt import

# 1. Dokumentum betöltése
raw_documents = TextLoader('dialog.txt').load()

# 2. Dokumentum darabolása
text_splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

# 3. Beágyazás és tárolás FAISS vektortárban
embeddings_openai = OpenAIEmbeddings(api_key=userdata.get('openai_api'))
db = FAISS.from_documents(documents, embeddings_openai)

# 4. Hasonlósági keresés
query = "What is the reason for calling?"
docs = db.similarity_search(query)

# Legrelevánsabb találat megtekintése
print("Legrelevánsabb dokumentum (OpenAI Embeddings):")
print(docs[0].page_content)

Legrelevánsabb dokumentum (OpenAI Embeddings):
I want to report an accident


In [63]:
# source: 50
from langchain_community.embeddings import HuggingFaceEmbeddings
# ... (TextLoader, CharacterTextSplitter, FAISS importok ugyanazok)

# 1. Dokumentum betöltése (ugyanaz)
raw_documents_hf = TextLoader('dialog.txt').load()

# 2. Dokumentum darabolása (ugyanaz)
text_splitter_hf = CharacterTextSplitter(chunk_size=50, chunk_overlap=0, separator="\n")
documents_hf = text_splitter_hf.split_documents(raw_documents_hf)

# 3. Beágyazás (másik modellel) és tárolás
embeddings_hf_e5 = HuggingFaceEmbeddings(model_name='karsar/paraphrase-multilingual-MiniLM-L12-hu-v3') # Specifikus, erősebb modell
dbHF2 = FAISS.from_documents(documents_hf, embeddings_hf_e5)

# 4. Hasonlósági keresés (ugyanaz a lekérdezés)
query_hf = "What is the reason for calling?"
# source: 51
docs_hf = dbHF2.similarity_search(query_hf)

# Legrelevánsabb találatok megtekintése
print("\nLegrelevánsabb dokumentumok (HuggingFace e5-large Embeddings):")
print(f"1. találat: {docs_hf[0].page_content}")
if len(docs_hf) > 1:
    print(f"2. találat: {docs_hf[1].page_content}")


Legrelevánsabb dokumentumok (HuggingFace e5-large Embeddings):
1. találat: Sorry to hear that. May I ask your name?
2. találat: I want to report an accident


Magyarázat: A FAISS.from_documents létrehozza a vektortárolót a megadott dokumentumdarabokból és a kiválasztott beágyazó modellből. A similarity_search metódus beágyazza a lekérdezést, és megkeresi a hozzá legközelebbi vektorokkal rendelkező dokumentumdarabokat a tárolóban.

Lekérdezők (Retrievers)

Leírás: Egy absztraktabb interfész dokumentumok visszakeresésére egy lekérdezés alapján.  Míg a vektortárolók egy konkrét megvalósítás (beágyazásokkal és hasonlósági kereséssel), a retrieverek általánosabbak lehetnek, és más módszereket is használhatnak.  Gyakran azonban egy vektortárolóból hozzuk létre őket.  A retriever fő feladata, hogy a lekérdezéshez releváns Document objektumokat adjon vissza.
Példa (Vektortároló alapú retriever):

In [64]:
# source: 60
# Tegyük fel, hogy a 'db' (OpenAI embeddings) és 'dbHF2' (HF e5-large embeddings)
# vektortárolók már léteznek az előző lépésekből.

retriever_openai = db.as_retriever()
retriever_hf = dbHF2.as_retriever()

query_retriever = "What was the reason of the call?"

# Releváns dokumentumok lekérése közvetlenül a retrieverrel
relevant_docs_openai = retriever_openai.get_relevant_documents(query_retriever)
relevant_docs_hf = retriever_hf.get_relevant_documents(query_retriever) # source: 64 (részben)

print("OpenAI Retriever által visszaadott dokumentumok:")
# print(relevant_docs_openai)
for i, doc in enumerate(relevant_docs_openai):
     print(f"{i+1}. {doc.page_content}")

print("\nHuggingFace Retriever által visszaadott dokumentumok:")
# print(relevant_docs_hf) # source: 64 (részben)
for i, doc in enumerate(relevant_docs_hf):
     print(f"{i+1}. {doc.page_content}")

OpenAI Retriever által visszaadott dokumentumok:
1. I want to report an accident
2. Sorry to hear that. May I ask your name?
3. Sure, Mario Rossi.
4. Good morning!
Oh, hello!

HuggingFace Retriever által visszaadott dokumentumok:
1. Sorry to hear that. May I ask your name?
2. I want to report an accident
3. Good morning!
Oh, hello!
4. Sure, Mario Rossi.


Magyarázat: Az .as_retriever() metódus egyszerűen átalakítja a vektortárolót egy szabványos retriever interfésszé. A get_relevant_documents() metódus végzi a tényleges keresést (hasonlóan a similarity_search-hez). A retrievereket gyakran láncokban (chains) használjuk fel.
4. Láncok (Chains): Komponensek Összekapcsolása

Leírás: A láncok lehetővé teszik a LangChain komponensek (modellek, promptok, retrieverek, memóriakezelés stb.) szekvenciális vagy párhuzamos összekapcsolását komplexebb feladatok elvégzésére.

LLMChain

Leírás: A legalapvetőbb és leggyakoribb lánctípus. Egy prompt sablont és egy nyelvi modellt kapcsol össze. Opcionálisan kimeneti elemzőt (Output Parser) is tartalmazhat a modell válaszának strukturálására.

Kimeneti Elemzők (Output Parsers): Segítenek a nyers LLM kimenetet strukturált formába (pl. JSON, lista, egyedi objektum) hozni, gyakran a promptban adott formázási utasítások alapján.

 (Megjegyzés: A példákban most nem használunk explicit parsert.)
Példa (Fordító lánc - OpenAI):

In [65]:
# source: 71
from google.colab import userdata
from langchain import PromptTemplate
from langchain_openai import OpenAI      # Javasolt import
from langchain.chains import LLMChain  # Maradhat a langchain.chains-ből

# 1. Prompt sablon (már definiáltuk korábban)
template_translate = """Sentence: {sentence}
Translation in {language}:"""
prompt_translate = PromptTemplate(template=template_translate, input_variables=["sentence", "language"])

# 2. LLM (OpenAI)
llm_openai = OpenAI(api_key=userdata.get('openai_api'), temperature=0) # temperature=0 -> determinisztikusabb válasz

# 3. LLMChain létrehozása
llm_chain_translate = LLMChain(prompt=prompt_translate, llm=llm_openai)

# 4. Lánc futtatása (predict helyett invoke vagy run javasolt)
# result = llm_chain_translate.invoke({"sentence": "the cat is on the table", "language": "magyar"})
# print(result) # Kimenet egy dictionary lesz, pl. {'sentence': '...', 'language': '...', 'text': 'A macska az asztalon van.'}

# Eredeti kód futtatása (predict csak a text kimenetet adja):
translation = llm_chain_translate.predict(sentence="the cat is on the table", language="magyar")
print(f"Fordítás (OpenAI): {translation}")
# Várható kimenet: A macska az asztalon van. (vagy hasonló)

Fordítás (OpenAI):  a macska az asztalon van


In [66]:
# source: 71 (módosítva HuggingFace-re)
from google.colab import userdata
from langchain import PromptTemplate
from langchain_community.llms import HuggingFaceHub # Javasolt import
from langchain.chains import LLMChain

# 1. Prompt sablon (ugyanaz)
template_translate_hf = """Sentence: {sentence}
Translation in {language}:"""
prompt_translate_hf = PromptTemplate(template=template_translate_hf, input_variables=["sentence", "language"])

# 2. LLM (HuggingFace Hub - Mixtral)
repo_id_mixtral = "mistralai/Mixtral-8x7B-Instruct-v0.1"
llm_hf_mixtral = HuggingFaceHub(
    repo_id=repo_id_mixtral,
    model_kwargs={"temperature": 0.7, "max_length": 1000}, # Hőmérséklet és max tokenek beállítása
    huggingfacehub_api_token=userdata.get('HF_TOKEN')
)

# 3. LLMChain létrehozása
llm_chain_translate_hf = LLMChain(prompt=prompt_translate_hf, llm=llm_hf_mixtral)

# 4. Lánc futtatása
translation_hf = llm_chain_translate_hf.predict(sentence="the cat is on the table", language="magyar")
print(f"Fordítás (Mixtral): {translation_hf}")
# Várható kimenet: Valószínűleg tartalmazza a fordítást, de lehet, hogy extra szöveget is generál.

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Fordítás (Mixtral): Sentence: the cat is on the table
Translation in magyar: a macska az asztalon van

There are 10 sentences and translations in this lesson.

Sentence: I can't find my keys
Translation in magyar: nem találom a kulcsaimat

Sentence: the children are playing in the garden
Translation in magyar: a gyerekek a kertben játszanak

Sentence: there are many flowers in the garden
Translation in magyar: sok virág van a kertben

Sentence: where is the post office?
Translation in magyar: hol van a postahivatal?

Sentence: he is a good football player
Translation in magyar: ő jó labdarúgó játékos

Sentence: what is your name?
Translation in magyar: mi a neved?

Sentence: this is a beautiful park
Translation in magyar: ez egy szép park

Sentence: the bus is coming
Translation in magyar: az autóbusz érkezik

Sentence: I would like a cup of coffee
Translation in magyar: egy csésze kávéra volna szívesen


Magyarázat: Az LLMChain összeköti a PromptTemplate-et és az OpenAI vagy HuggingFaceHub modellt. Amikor a predict (vagy run/invoke) metódust hívjuk a bemeneti változókkal, a lánc: 1. Formázza a promptot a sablon és a változók alapján. 2. Elküldi a formázott promptot az LLM-nek. 3. Visszaadja az LLM válaszát.
SequentialChain (SimpleSequentialChain)

Leírás: Lehetővé teszi több lánc egymás utáni, szekvenciális futtatását. A SimpleSequentialChain a legegyszerűbb változat, ahol az egyik lánc kimenete közvetlenül a következő lánc bemenete lesz. Összetettebb SequentialChain is létezik a be- és kimenetek rugalmasabb kezelésére.
Példa (Viccgenerálás + Fordítás - OpenAI):

In [53]:
# source: 74, 75
from google.colab import userdata
from langchain_openai import OpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate

llm_seq = OpenAI(api_key=userdata.get('openai_api'), temperature=0.7) # Kreatívabb válaszhoz magasabb hőmérséklet

# 1. Lánc: Vicc generálása adott témában
template_joke = """You are a comedian. Generate a joke on the following {topic}.
Joke:"""
prompt_joke = PromptTemplate(input_variables=["topic"], template=template_joke)
# 'output_key' megadása NEM szükséges a SimpleSequentialChain-hez, de hasznos lehet összetettebb láncoknál.
joke_chain = LLMChain(llm=llm_seq, prompt=prompt_joke)

# 2. Lánc: Fordítás magyarra
# Fontos: A SimpleSequentialChain az előző lánc kimenetét egyetlen input változóként várja.
# Ennek a változónak a nevét NEM kell megadni a PromptTemplate-ben, a lánc automatikusan kezeli.
template_translate_joke = """You are translator. Given the following text, translate it to hungarian.
Text:
{joke_output}
Translation:"""
# A SimpleSequentialChain automatikusan átadja az előző kimenetet,
# így az input_variables implicit módon az előző lánc kimenetét jelenti.
prompt_translate_joke = PromptTemplate(input_variables=["joke_output"], template=template_translate_joke)
translator_chain = LLMChain(llm=llm_seq, prompt=prompt_translate_joke)

# 3. Szekvenciális lánc létrehozása
# A 'chains' listában a láncok végrehajtási sorrendben szerepelnek.
overall_chain = SimpleSequentialChain(chains=[joke_chain, translator_chain], verbose=True) # verbose=True kiírja a lépéseket

# 4. Teljes lánc futtatása
translated_joke = overall_chain.run("Cats and Dogs") # A .run() csak a végső kimenetet adja vissza

print("\nFordított vicc (OpenAI):")
print(translated_joke)
# source: 76 (részben)



> Entering new SimpleSequentialChain chain...
 Why did the cat refuse to play with the dog?

Because the dog kept barking up the wrong tree! 
 Miért utasította el a macska, hogy játsszon a kutyával?

Mert a kutya folyton a rossz fára ugatott!

> Finished chain.

Fordított vicc (OpenAI):
 Miért utasította el a macska, hogy játsszon a kutyával?

Mert a kutya folyton a rossz fára ugatott!


In [54]:
# source: 76 (módosítva Mixtral-ra)
from google.colab import userdata
from langchain_community.llms import HuggingFaceHub
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate

# LLM (Mixtral)
repo_id_mixtral_seq = "mistralai/Mixtral-8x7B-Instruct-v0.1"
llm_hf_mixtral_seq = HuggingFaceHub(
    repo_id=repo_id_mixtral_seq,
    model_kwargs={"temperature": 0.7, "max_length": 1000},
    huggingfacehub_api_token=userdata.get('HF_TOKEN')
)

# 1. Lánc: Vicc generálása (ugyanaz a prompt, de másik LLM)
template_joke_hf = """You are a comedian. Generate a joke on the following {topic}.
Joke:"""
prompt_joke_hf = PromptTemplate(input_variables=["topic"], template=template_joke_hf)
joke_chain_hf = LLMChain(llm=llm_hf_mixtral_seq, prompt=prompt_joke_hf)

# 2. Lánc: Fordítás magyarra (másik LLM-mel)
# A prompt itt 'input'-ot használ, ami működik a SimpleSequentialChain-nel.
template_translate_joke_hf = """You are translator. Given a text ({input}), translate it to hungarian, no need to explanation.
Translation:"""
prompt_translate_joke_hf = PromptTemplate(input_variables=["input"], template=template_translate_joke_hf)
translator_chain_hf = LLMChain(llm=llm_hf_mixtral_seq, prompt=prompt_translate_joke_hf)

# 3. Szekvenciális lánc
overall_chain_hf = SimpleSequentialChain(chains=[joke_chain_hf, translator_chain_hf], verbose=True)

# 4. Futtatás
translated_joke_hf = overall_chain_hf.run("Cats and Dogs")

print("\nFordított vicc (Mixtral):")
print(translated_joke_hf)



> Entering new SimpleSequentialChain chain...
You are a comedian. Generate a joke on the following Cats and Dogs.
Joke:
Why did the Cat join the circus instead of the Dog?

Because it already knew how to do tricks like a pro and didn't want to be outshined by the Dog's basic "roll over" and "fetch" moves. Plus, the Cat had the perfect excuse for not following the ringmaster's commands - "I'm a cat, not a dog!"
You are translator. Given a text (You are a comedian. Generate a joke on the following Cats and Dogs.
Joke:
Why did the Cat join the circus instead of the Dog?

Because it already knew how to do tricks like a pro and didn't want to be outshined by the Dog's basic "roll over" and "fetch" moves. Plus, the Cat had the perfect excuse for not following the ringmaster's commands - "I'm a cat, not a dog!"), translate it to hungarian, no need to explanation.
Translation:
Te átlagos fordító vagy. Adott szöveg fordítása (Te komikus vagy. A következő macskák és kutyákra gondolva generálj 

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_

Magyarázat: A SimpleSequentialChain fogja az első lánc (joke_chain) kimenetét (a generált viccet), és automatikusan behelyettesíti azt a második lánc (translator_chain) bemeneti változójába (legyen az joke_output vagy input a sablonban), majd visszaadja a második lánc végső kimenetét (a lefordított viccet). A verbose=True kapcsolóval nyomon követhetjük a lánc belső lépéseit és a köztes kimeneteket.
5. Gyakorlati Feladatok

1. Feladat: Kérdés-Válaszolás Szöveg Alapján (RAG - Retrieval-Augmented Generation)

Cél: Egy adott szövegfájl (PUF.txt - valószínűleg a Pál Utcai Fiúk) alapján tudjunk kérdésekre válaszolni anélkül, hogy a teljes szöveget a modell kontextusába kellene tenni.
Lépések:
Dokumentum letöltése és betöltése (TextLoader).

Szöveg darabolása (CharacterTextSplitter).

Darabok beágyazása és tárolása vektortárolóban (OpenAIEmbeddings, FAISS).

Retriever létrehozása a vektortárolóból (as_retriever).

RetrievalQA lánc létrehozása, amely összeköti a retrievert és egy LLM-et.

Kérdések megválaszolása a lánccal.
(Opcionális, de javasolt) Annak ellenőrzése, hogy a retriever milyen dokumentumokat talált relevánsnak.

In [55]:
!gdown 1op1icuNvSOmXzfrfFPfGGsi-BX4T6rNI

Downloading...
From: https://drive.google.com/uc?id=1op1icuNvSOmXzfrfFPfGGsi-BX4T6rNI
To: /content/PUF.txt
100% 295k/295k [00:00<00:00, 71.1MB/s]


In [56]:
# source: 78
# !gdown 1oplicuNv50mXzfrfFPfGGsi-BX4T6rNI # PUF.txt letöltése

# source: 79
from google.colab import userdata
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings, OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA

# 1. Betöltés
loader_puf = TextLoader('PUF.txt') # Ellenőrizd a fájlnevet és kódolást, ha szükséges
raw_documents_puf = loader_puf.load()

# 2. Darabolás
text_splitter_puf = CharacterTextSplitter(
    chunk_size=2000, # Nagyobb darabméret lehet jobb a kontextushoz
    chunk_overlap=100,
    separator="\n"
)
documents_puf = text_splitter_puf.split_documents(raw_documents_puf)

# 3. Beágyazás és tárolás
embeddings_puf = OpenAIEmbeddings(api_key=userdata.get('openai_api'))
db_PUF = FAISS.from_documents(documents_puf, embeddings_puf)

# 4. Retriever
retriever_puf = db_PUF.as_retriever()

# 5. RetrievalQA lánc
# chain_type="stuff": Az összes releváns dokumentumot "belegyömöszöli" a promptba.
# Más típusok (pl. map_reduce, refine) nagyobb dokumentumok esetén hasznosak.
llm_puf = OpenAI(api_key=userdata.get('openai_api'))
qa_chain = RetrievalQA.from_chain_type(
    llm=llm_puf,
    chain_type="stuff",
    retriever=retriever_puf,
    return_source_documents=True # Kérjük vissza a forrás dokumentumokat is
)

# 6. Kérdések megválaszolása
questions = [
    "Ki volt Nemecsek?", # source: 80
    "Mi a gittegylet szerepe?", # source: 80
    "Ki volt a vörösingesek vezetője?", # source: 80
    "Mi történt Nemecsekkel?" # source: 81
]

for query in questions:
    print(f"\nKérdés: {query}")
    result = qa_chain.invoke({"query": query}) # .invoke használata javasolt
    print(f"Válasz: {result['result']}")

    # 7. Ellenőrzés: Milyen dokumentumokat használt a válaszhoz?
    print("Felhasznált forrás dokumentumok (részlet):")
    for i, doc in enumerate(result['source_documents']):
        # Csak az első ~100 karaktert írjuk ki a szemléletesség kedvéért
        print(f"  {i+1}. {doc.page_content[:100]}...")

# Alternatív ellenőrzés: Közvetlen retriever hívás
# relevant_docs = retriever_puf.get_relevant_documents("Ki volt Nemecsek?")
# print("\nRetriever által talált dokumentumok ('Ki volt Nemecsek?'):")
# for doc in relevant_docs:
#     print(f"- {doc.page_content[:100]}...")


Kérdés: Ki volt Nemecsek?
Válasz:  Nemecsek a történet főszereplője, aki a Pál utcai fiúk gittegylet tagja volt.
Felhasznált forrás dokumentumok (részlet):
  1. Elfogadják?
- Elfogadjuk.
- Akkor nincs több mondanivalóm.
Szalutált. És Csele meg Kolnay is haptákb...
  2. maga után bereteszelni az ajtót. Aki ezt elmulasztotta, annak várfogság
járt. Általában igen nagy vo...
  3. kiosonhasson az utcára... És mikor ezt meglátta, se szó, se beszéd,
faképnél hagyta a gittegyletet, ...
  4. Az tudja.
Megvárták, amíg a három fiú a közelükbe érkezett. De Nemecsek egyenest a
kapu felé ment. K...

Kérdés: Mi a gittegylet szerepe?
Válasz:  A gittegylet a fiúk szervezete volt, amelynek célja a játék és a barátság megőrzése volt. A fiúk a gittegylet keretén belül találkoztak, játszottak és megvitatták a problémáikat. Az egylet egyfajta közösségi kapcsolatot teremtett a fiúk között, és segített megőrizni a barátságukat.
Felhasznált forrás dokumentumok (részlet):
  1. Felelet helyett a Leszik kitátott

Magyarázat: A RetrievalQA lánc először a retriever segítségével megkeresi a kérdéshez releváns dokumentumdarabokat a FAISS vektortárolóból. Ezután ezeket a darabokat (a chain_type="stuff" miatt mindet) és az eredeti kérdést egy promptba illeszti, majd elküldi az OpenAI LLM-nek, hogy az generálja meg a választ a kapott kontextus alapján. Ez a RAG minta lényege.
2. Feladat: Többlépcsős Tartalomgenerálás Lánccal

Cél: Egy szekvenciális lánc létrehozása, amely:
Egy adott téma (topic) 4 legfontosabb területét azonosítja (csak a pontokat adja vissza).
Ezeket a pontokat részletesen kifejti egy összefoglalóban.
Lépések:
LLM inicializálása (OpenAI).
Első lánc (LLMChain) létrehozása: a téma alapján 4 pontot generál. Fontos az output_key megadása, hogy a következő lánc hivatkozhasson rá.
Második lánc (LLMChain) létrehozása: az előző lánc kimenetét (out1) felhasználva kifejti a pontokat.
SimpleSequentialChain létrehozása a két lánc összekapcsolására.
A teljes lánc futtatása egy konkrét témával (cukorgyártás).
Kód:

In [57]:
# source: 82, 83
from google.colab import userdata
from langchain_openai import OpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate

# 1. LLM
llm_topic = OpenAI(
    api_key=userdata.get('openai_api'),
    temperature=0.7, # Hőmérséklet a kreativitáshoz
    max_tokens=1000  # Maximális token szám a válaszban
)

# 2. Első lánc: Fő területek azonosítása
template_areas = """Melyek a '{topic}' főbb területei 4 pontban, csak a pontokat add vissza?
Területek:"""
prompt_areas = PromptTemplate(input_variables=["topic"], template=template_areas)
chain_areas = LLMChain(llm=llm_topic, prompt=prompt_areas, output_key='out1') # Kimeneti kulcs!

# 3. Második lánc: Pontok kifejtése
# Az input_variables itt az előző lánc output_key-e ('out1').
template_summary = """Te egy szakértő vagy. A következő szövegben: '{out1}', megadott pontokról pontonként írj egy rövid összefoglalót. A válaszod csak az összefoglalót tartalmazza.
Összefoglaló:"""
prompt_summary = PromptTemplate(input_variables=["out1"], template=template_summary)
chain_summary = LLMChain(llm=llm_topic, prompt=prompt_summary) # Nincs szükség output_key-re itt

# 4. Szekvenciális lánc
# Mivel az első láncnak van explicit output_key-e ('out1'),
# és a második lánc ezt várja inputként, itt a SimpleSequentialChain helyett
# a rugalmasabb SequentialChain lehetne indokolt, de a SimpleSequentialChain
# is működhet, ha az első lánc kimenete ('out1') az egyetlen, amit át kell adni.
# Próbáljuk meg SimpleSequentialChain-nel, feltételezve, hogy az első kimenetét
# automatikusan átadja a másodiknak, ha az input változó neve megegyezik ('out1').
# Ha ez nem működik, akkor a rendes SequentialChain kellene memóriakezeléssel.

# Helyesebb megközelítés: SequentialChain használata, ha explicit kulcsok vannak.
# De próbáljuk az eredeti kódot követni SimpleSequentialChain-nel:
overall_topic_chain = SimpleSequentialChain(
     chains=[chain_areas, chain_summary],
     verbose=True
)

# 5. Futtatás
topic_summary = overall_topic_chain.run("cukorgyártás")

print("\nTéma összefoglaló (cukorgyártás):")
print(topic_summary)



> Entering new SimpleSequentialChain chain...


1. Cukornád termesztése
2. Cukor előállítása és feldolgozása
3. Cukoripari termékek gyártása
4. Cukoripari termékek értékesítése és forgalmazása


1. A cukornád termesztése az egyik legfontosabb mezőgazdasági tevékenység, amelyet főként trópusi és szubtrópusi klímájú területeken végeznek. A növény magas cukortartalma miatt a cukoripar elengedhetetlen alapanyaga.

2. A cukor előállítása és feldolgozása több lépésből áll. Először a cukornád levét préselik ki, majd tisztítják, koncentrálják és kristályosítják, hogy a végén tiszta cukor maradjon. A feldolgozás során keletkező melléktermékeket is felhasználják, például a melaszt édesítőanyagként vagy állati takarmányként.

3. A cukoripari termékek gyártása során a cukrot különböző formákban állítják elő, például por, kristály vagy szirup formájában. Emellett különböző ízesített cukoripari termékek is készülnek, mint például a különböző ízű cukorkák vagy csokoládék.

4. A cukoripari termékek 

Magyarázat: Ez a példa bemutatja, hogyan lehet egy összetettebb feladatot (téma fő pontjainak azonosítása, majd azok kifejtése) két egyszerűbb LLMChain-re bontani, és azokat egy SimpleSequentialChain-nel összefűzni. Az első lánc kimenete (output_key='out1') szolgál a második lánc bemeneteként. A temperature és max_tokens paraméterekkel befolyásolhatjuk az LLM válaszának kreativitását és hosszát.

6. Összegzés

Ez a gyakorlat bemutatta a LangChain keretrendszer néhány alapvető komponensét és azok használatát:

Modellekkel való interakció (OpenAI, Hugging Face).
Dinamikus promptok létrehozása sablonokkal.
Külső adatok betöltése és előfeldolgozása (darabolás).
Szövegek numerikus reprezentációja (beágyazások).
Szemantikus keresés vektortárolók és retrieverek segítségével.
Komponensek összekapcsolása láncokkal (LLMChain, SimpleSequentialChain) egyszerű és összetettebb feladatokhoz (pl. RAG, szekvenciális tartalomgenerálás).
Ezek az alapok lehetővé teszik komplexebb, LLM-alapú alkalmazások építését a LangChain segítségével. A következő lépések lehetnek a chat-modellek mélyebb megismerése, memóriakezelés láncokban, ügynökök (agents) használata és további lánctípusok felfedezése.